In [1]:
import os
import sys

import pandas as pd
import numpy as np
import sklearn
from hmmlearn import hmm
try: # version > 0.2.7
   from hmmlearn.hmm import CategoricalHMM as MultinomialHMM
except: # version <= 0.2.7
   from hmmlearn.hmm import MultinomialHMM
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns
from sklearn.model_selection import KFold

sys.path.insert(0, '../scripts')
from flaHMM_functions import *

In [2]:
species_build=['Dyak.GCF_016746365',
              'Dsan.GCF_016746245',
              'Dsim.GCF_016746395',
              'Dmau.GCF_004382145',
              'Dmel.dm6',
              'Dsubp.GCF_014743375']

In [3]:
for bin_size in tqdm(['10','5','2.5']):
#for bin_size in tqdm(['10']):
    all_data_species=pd.DataFrame()
    for species_train in species_build:
        print(species_train)
        species_train_build='_build_'.join(species_train.split('.'))
        all_data=read_proces_05files(species_train,'../bins/bins_'+bin_size+'k/')
    
        try:
            all_data=genome_cluster_annotation(species_train_build,all_data)
        except:
            all_data['cluster']=0
        
        all_data = all_data[['chr', 'bin_start','bin_end','cluster','coverage_plus','coverage_minus']]
        all_data['Data']=species_train

        centromere_2_bins=pd.DataFrame()
        for chromosome in ['chr2L','chr2R']:
            get_centromere_coordinates=centromere_coordinates[(centromere_coordinates['species']==species_train)&(centromere_coordinates['chr']==chromosome)]
            for row in get_centromere_coordinates.iterrows():
                centromere_2_bins=pd.concat([centromere_2_bins, all_data[(all_data['chr']==chromosome)&(all_data['bin_start']>=row[1]['bin_start'])&(all_data['bin_end']<=row[1]['bin_end'])]])
            
        centromere_3_bins=pd.DataFrame()
        for chromosome in ['chr3L','chr3R']:
            get_centromere_coordinates=centromere_coordinates[(centromere_coordinates['species']==species_train)&(centromere_coordinates['chr']==chromosome)]
            for row in get_centromere_coordinates.iterrows():
                centromere_3_bins=pd.concat([centromere_3_bins, all_data[(all_data['chr']==chromosome)&(all_data['bin_start']>=row[1]['bin_start'])&(all_data['bin_end']<=row[1]['bin_end'])]])
          
        flam_bins=all_data[all_data['cluster']==1]
        print(flam_bins.shape)
    
        region=[]
        region_binary=[]
        for i in all_data.index.tolist():
            if i in flam_bins.index.tolist():
                region.append('flam')
                region_binary.append(1)
            elif i in centromere_2_bins.index.tolist():
                region.append('centromere')
                region_binary.append(2)
            elif i in centromere_3_bins.index.tolist():
                region.append('centromere')
                region_binary.append(2)
            else:
                region.append('none')
                region_binary.append(0)
        
        all_data['region']=region
        all_data['region_binary']=region_binary
        
        all_data_species=pd.concat([all_data_species, all_data])
        
    all_data_species.to_csv('all_data_species'+bin_size+'k.txt', sep='\t')

  0%|          | 0/3 [00:00<?, ?it/s]

Dyak.GCF_016746365
(284, 7)
Dsan.GCF_016746245
(165, 7)
Dsim.GCF_016746395
(90, 7)
Dmau.GCF_004382145
(171, 7)
Dmel.dm6
(162, 7)
Dsubp.GCF_014743375
(219, 7)


 33%|███▎      | 1/3 [00:56<01:52, 56.41s/it]

Dyak.GCF_016746365
(283, 7)
Dsan.GCF_016746245
(164, 7)
Dsim.GCF_016746395
(89, 7)
Dmau.GCF_004382145
(170, 7)
Dmel.dm6
(161, 7)
Dsubp.GCF_014743375
(218, 7)


 67%|██████▋   | 2/3 [01:49<00:54, 54.47s/it]

Dyak.GCF_016746365
(566, 7)
Dsan.GCF_016746245
(326, 7)
Dsim.GCF_016746395
(176, 7)
Dmau.GCF_004382145
(339, 7)
Dmel.dm6
(322, 7)
Dsubp.GCF_014743375
(435, 7)


100%|██████████| 3/3 [04:10<00:00, 83.55s/it]
